In [33]:
import numpy as np
import tensorflow as tf
import os

In [34]:
import matplotlib.pyplot as plt
import _pickle
import time

In [35]:
# todo
# compare likelihoods
# test with less popular features
# use different priors
# multiclass logistic regression significance tests
# fit mixture of gaussian vs gaussian
# check if weight activity killed
# tansfer learning for new class and robustness

In [36]:
turker_count= {'Alec Baldwin': {'beard': 7,
  'cheek': 48,
  'ears': 14,
  'eye brows': 67,
  'eyes': 190,
  'hairline': 37,
  'lips': 120,
  'moustache': 8,
  'nose': 111},
 'America Ferrera': {'beard': 1,
  'cheek': 103,
  'ears': 4,
  'eye brows': 68,
  'eyes': 110,
  'hairline': 22,
  'lips': 191,
  'moustache': 4,
  'nose': 99},
 'Angie Harmon': {'beard': 4,
  'cheek': 93,
  'ears': 5,
  'eye brows': 94,
  'eyes': 149,
  'hairline': 20,
  'lips': 160,
  'moustache': 5,
  'nose': 72},
 'Anne Hathaway': {'beard': 0,
  'cheek': 30,
  'ears': 4,
  'eye brows': 65,
  'eyes': 182,
  'hairline': 34,
  'lips': 238,
  'moustache': 0,
  'nose': 49},
 'Bill Hader': {'beard': 6,
  'cheek': 48,
  'ears': 9,
  'eye brows': 91,
  'eyes': 197,
  'hairline': 57,
  'lips': 109,
  'moustache': 8,
  'nose': 77},
 'Cheryl Hines': {'beard': 2,
  'cheek': 61,
  'ears': 8,
  'eye brows': 32,
  'eyes': 163,
  'hairline': 38,
  'lips': 219,
  'moustache': 1,
  'nose': 78},
 'Daniel Radcliffe': {'beard': 9,
  'cheek': 47,
  'ears': 10,
  'eye brows': 125,
  'eyes': 193,
  'hairline': 51,
  'lips': 89,
  'moustache': 8,
  'nose': 70},
 'Fran Drescher': {'beard': 0,
  'cheek': 72,
  'ears': 4,
  'eye brows': 86,
  'eyes': 140,
  'hairline': 25,
  'lips': 207,
  'moustache': 7,
  'nose': 61},
 'Gerard Butler': {'beard': 125,
  'cheek': 27,
  'ears': 14,
  'eye brows': 41,
  'eyes': 194,
  'hairline': 18,
  'lips': 59,
  'moustache': 49,
  'nose': 75},
 'Jennifer Aniston': {'beard': 3,
  'cheek': 59,
  'ears': 13,
  'eye brows': 48,
  'eyes': 203,
  'hairline': 74,
  'lips': 140,
  'moustache': 1,
  'nose': 61},
 'Kristin Chenoweth': {'beard': 3,
  'cheek': 98,
  'ears': 6,
  'eye brows': 48,
  'eyes': 175,
  'hairline': 40,
  'lips': 174,
  'moustache': 4,
  'nose': 54},
 'Lorraine Bracco': {'beard': 4,
  'cheek': 100,
  'ears': 11,
  'eye brows': 40,
  'eyes': 131,
  'hairline': 54,
  'lips': 130,
  'moustache': 1,
  'nose': 131},
 'Matt Damon': {'beard': 9,
  'cheek': 55,
  'ears': 15,
  'eye brows': 43,
  'eyes': 166,
  'hairline': 47,
  'lips': 151,
  'moustache': 9,
  'nose': 107},
 'Michael Vartan': {'beard': 80,
  'cheek': 27,
  'ears': 17,
  'eye brows': 37,
  'eyes': 145,
  'hairline': 53,
  'lips': 64,
  'moustache': 37,
  'nose': 142},
 'Nicolas Cage': {'beard': 13,
  'cheek': 44,
  'ears': 24,
  'eye brows': 74,
  'eyes': 178,
  'hairline': 44,
  'lips': 99,
  'moustache': 10,
  'nose': 116},
 'Selena Gomez': {'beard': 1,
  'cheek': 126,
  'ears': 4,
  'eye brows': 32,
  'eyes': 165,
  'hairline': 35,
  'lips': 182,
  'moustache': 6,
  'nose': 51},
 'Steve Carell': {'beard': 21,
  'cheek': 46,
  'ears': 26,
  'eye brows': 105,
  'eyes': 123,
  'hairline': 26,
  'lips': 99,
  'moustache': 17,
  'nose': 139}}

In [37]:
file = open('true_class_saliency_mask_vanilla_temp1_10.pkl', "rb")
# file = open('mean_diff_saliency_mask_vanilla_temp1.pkl', "rb")
true_saliencies = _pickle.load(file) 
file = open('mean.pkl', "rb")
gaussian_means = _pickle.load(file) 
file = open('covariance.pkl', "rb")
gaussian_covariances = _pickle.load(file)

with open('turker_prob.pkl', "rb") as handle:
    turkers = _pickle.load(handle) 

with open('test_set_17_227.pkl', "rb") as handle:
    test_set = _pickle.load(handle) 
# with open('turker_count.pkl', "rb") as handle:
#     turker_count= _pickle.load(handle) 
# with open('top_2_features_2.pkl', "rb") as handle:
#     turker_count= _pickle.load(handle) 

In [38]:
true_saliencies['Alec Baldwin'].shape

(15, 227, 227)

In [39]:
male_actors = ['Daniel Radcliffe', 'Gerard Butler', 'Michael Vartan', 'Alec Baldwin', 'Steve Carell',\
                    'Bill Hader', 'Fran Drescher', 'Matt Damon', 'Nicolas Cage']
female_actors = ['Cheryl Hines', 'Selena Gomez', 'Angie Harmon', 'Lorraine Bracco', 'Kristin Chenoweth',\
                      'America Ferrera', 'Anne Hathaway', 'Jennifer Aniston']
gender ={}
for actor in male_actors:
    gender[actor] = 'male'
for actor in female_actors:
    gender[actor] = 'female' 

In [40]:
# creating turker coiunt per actor
features = ['beard', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
actor_stats = {}
for actor in test_set:
    actor_stats[actor] = {}
    for feature in features:
        actor_stats[actor][feature] = 0  
#     for stats in turker_count[actor]:
#         for feature in features:
#             try:
#                  actor_stats[actor][feature] += stats[feature]
#             except:
#                 continue
                
    stats = turker_count[actor]
    for feature in features:
        try:
             actor_stats[actor][feature] += stats[feature]
        except:
            continue

actor_stats_normalized = {}
for actor in actor_stats:
    actor_stats_normalized[actor] = {}
    total_count = sum(actor_stats[actor].values())
    for feature in features:
        actor_stats_normalized[actor][feature] = actor_stats[actor][feature]/total_count

In [41]:
turker_count

{'Alec Baldwin': {'beard': 7,
  'cheek': 48,
  'ears': 14,
  'eye brows': 67,
  'eyes': 190,
  'hairline': 37,
  'lips': 120,
  'moustache': 8,
  'nose': 111},
 'America Ferrera': {'beard': 1,
  'cheek': 103,
  'ears': 4,
  'eye brows': 68,
  'eyes': 110,
  'hairline': 22,
  'lips': 191,
  'moustache': 4,
  'nose': 99},
 'Angie Harmon': {'beard': 4,
  'cheek': 93,
  'ears': 5,
  'eye brows': 94,
  'eyes': 149,
  'hairline': 20,
  'lips': 160,
  'moustache': 5,
  'nose': 72},
 'Anne Hathaway': {'beard': 0,
  'cheek': 30,
  'ears': 4,
  'eye brows': 65,
  'eyes': 182,
  'hairline': 34,
  'lips': 238,
  'moustache': 0,
  'nose': 49},
 'Bill Hader': {'beard': 6,
  'cheek': 48,
  'ears': 9,
  'eye brows': 91,
  'eyes': 197,
  'hairline': 57,
  'lips': 109,
  'moustache': 8,
  'nose': 77},
 'Cheryl Hines': {'beard': 2,
  'cheek': 61,
  'ears': 8,
  'eye brows': 32,
  'eyes': 163,
  'hairline': 38,
  'lips': 219,
  'moustache': 1,
  'nose': 78},
 'Daniel Radcliffe': {'beard': 9,
  'cheek': 47

In [42]:
actor_stats_normalized

{'Alec Baldwin': {'beard': 0.012962962962962963,
  'eye brows': 0.12407407407407407,
  'eyes': 0.35185185185185186,
  'hairline': 0.06851851851851852,
  'lips': 0.2222222222222222,
  'moustache': 0.014814814814814815,
  'nose': 0.20555555555555555},
 'America Ferrera': {'beard': 0.00202020202020202,
  'eye brows': 0.13737373737373737,
  'eyes': 0.2222222222222222,
  'hairline': 0.044444444444444446,
  'lips': 0.38585858585858585,
  'moustache': 0.00808080808080808,
  'nose': 0.2},
 'Angie Harmon': {'beard': 0.007936507936507936,
  'eye brows': 0.1865079365079365,
  'eyes': 0.29563492063492064,
  'hairline': 0.03968253968253968,
  'lips': 0.31746031746031744,
  'moustache': 0.00992063492063492,
  'nose': 0.14285714285714285},
 'Anne Hathaway': {'beard': 0.0,
  'eye brows': 0.11443661971830986,
  'eyes': 0.3204225352112676,
  'hairline': 0.05985915492957746,
  'lips': 0.41901408450704225,
  'moustache': 0.0,
  'nose': 0.08626760563380281},
 'Bill Hader': {'beard': 0.011009174311926606,
 

In [43]:
def create_individual_prior(test_subject):

    if gender[test_subject] == 'male':
        actor_set = male_actors.copy()
    else:
        actor_set = female_actors.copy()
    
    actor_set = male_actors.copy() + female_actors.copy()
    actor_set.remove(test_subject)
    prior = {}
    for feature in features:
        feature_stats = [actor_stats_normalized[actor][feature] for actor in actor_set]
        feature_prior = sum(feature_stats)/len(feature_stats)
        prior[feature] = feature_prior
    
    return prior

In [44]:
# monte carlo rectangles generation
def generate_rectangles(means, covariances):
    mc_samples = {}
    for feature in features:
#         print(feature)
        rectangles = []
        while len(rectangles) < 100:
#             print(len(rectangles))
            width, height, y_centre, x_centre = np.random.multivariate_normal(means[feature], \
                                                                              covariances[feature])
            x_0= int(np.round(x_centre - width/2)) 
            x_1= int(np.round(x_centre + width/2)) 
            y_0= int(np.round(226 - y_centre - height/2)) 
            y_1= int(np.round(226 - y_centre + height/2)) 
            
#             x_0= int(np.round(x_centre - height/2)) 
#             x_1= int(np.round(x_centre + height/2)) 
#             y_0= int(np.round(y_centre - width/2)) 
#             y_1= int(np.round(y_centre + width/2)) 
            
#             print(np.array([x_centre, y_centre, height, width]))
#             print(np.array([x_0,x_1,y_0,y_1]))
            if 0<= x_0 < x_1 <=226 and 0<= y_0 < y_1 <=226:
                rectangles.append(np.array([x_0,x_1,y_0,y_1]))
                
        mc_samples[feature] = np.stack(rectangles)
        
    return mc_samples 

mc_rectangle_samples = generate_rectangles(gaussian_means, gaussian_covariances)

In [45]:
def likelihood(rectangles, saliency_map):
    scores = {}
    for feature in features:
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
            area = (rec[3]-rec[2])*(rec[1]-rec[0])
            intensity_ratio = (np.sum(highlight))/np.sum(saliency_map)/(227**2)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in features:
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [46]:
def posterior(rectangles, saliency_map, test_subject):
    scores = {}
    prior = create_individual_prior(test_subject)
    for feature in features:
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
            area = (rec[3]-rec[2])*(rec[1]-rec[0])
            intensity_ratio = (np.sum(highlight))/np.sum(saliency_map)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = prior[feature]*intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in sorted(scores.keys()):
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [47]:
actor_true_saliencies = {}

for actor in true_saliencies:
    actor_saliency = np.mean(true_saliencies[actor], axis=0)
    actor_true_saliencies[actor] = actor_saliency

In [48]:
predictions = {}
for actor in sorted(test_set.keys()):
    saliency_map = actor_true_saliencies[actor]
    probs = likelihood(mc_rectangle_samples, saliency_map)
    predictions[actor]=probs

In [49]:
posterior_predictions = {}
for actor in sorted(test_set.keys()):
    saliency_map = actor_true_saliencies[actor]
    probs = posterior(mc_rectangle_samples, saliency_map, actor)
    posterior_predictions[actor]=probs

In [50]:
# main problems are with turker data
# maybe imbalanced, also tend to label similar feature
# maybe should try new set of actors who have very distinct features to do anchoring
# people choocse what seems to get highlighted in that image, but not what is salient about the person
# probaly better to frame questionas as what is salient about this individual

In [51]:
import operator

actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 4, figsize=(60,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_true_saliencies[actor])
        
    prediction = posterior_predictions[actor]
    turker_data = actor_stats_normalized[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].set_ylim([0, 0.7])
    ax[1].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('turkers vs posterior', fontsize=17)
    
    
    prediction = create_individual_prior(actor)
    X = np.arange(len(turker_data))
    ax[2].set_ylim([0, 0.7])
    ax[2].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('turkers vs prior', fontsize=17)
    
    prediction = predictions[actor]
    X = np.arange(len(turker_data))
    ax[3].set_ylim([0, 0.7])
    ax[3].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Backprop Prediction')
    ax[3].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[3].legend()
    ax[3].set_xticks(X)
    ax[3].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[3].set_title('turkers vs backprop', fontsize=17)

    print(actor)
    
    
plt.savefig('genderless_robust_predictions_non_normalized.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [52]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = posterior_predictions[actor]
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=1.0, pvalue=0.0)
America Ferrera
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Angie Harmon
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Anne Hathaway
SpearmanrResult(correlation=0.9189562119494703, pvalue=0.0034366261562175319)
Bill Hader
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Cheryl Hines
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Daniel Radcliffe
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Fran Drescher
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Gerard Butler
SpearmanrResult(correlation=0.32142857142857151, pvalue=0.4820720382996776)
Jennifer Aniston
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Kristin Chenoweth
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Lorrain

In [53]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = create_individual_prior(actor)
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
America Ferrera
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Angie Harmon
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Anne Hathaway
SpearmanrResult(correlation=0.9189562119494703, pvalue=0.0034366261562175319)
Bill Hader
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Cheryl Hines
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Daniel Radcliffe
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Fran Drescher
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Gerard Butler
SpearmanrResult(correlation=0.42857142857142866, pvalue=0.33736831108582399)
Jennifer Aniston
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Kristin Chenoweth
SpearmanrResult(correlation=0.96428571428571452, pvalue=

In [54]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = predictions[actor]
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.67857142857142871, pvalue=0.093750253959831303)
America Ferrera
SpearmanrResult(correlation=0.78571428571428592, pvalue=0.036238462679827131)
Angie Harmon
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Anne Hathaway
SpearmanrResult(correlation=0.90093746269555897, pvalue=0.0056205867153450136)
Bill Hader
SpearmanrResult(correlation=0.78571428571428592, pvalue=0.036238462679827131)
Cheryl Hines
SpearmanrResult(correlation=0.67857142857142871, pvalue=0.093750253959831303)
Daniel Radcliffe
SpearmanrResult(correlation=0.82142857142857151, pvalue=0.023448808345691505)
Fran Drescher
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Gerard Butler
SpearmanrResult(correlation=0.42857142857142866, pvalue=0.33736831108582399)
Jennifer Aniston
SpearmanrResult(correlation=0.75000000000000022, pvalue=0.052181400457057762)
Kristin Chenoweth
SpearmanrResult(correlation=0.71428571428571441, pvalue=0.0713